In [0]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import os

class DepthmapSet(Dataset):
    """ 
    A customized dataloader for the depth map dataset

    """
    def __init__(self, images, labels):
        """ Initialize the dataloader. We transform the np arrays 
        to torch tensors and also floats 
        Input:
          images: (N, 200, 200) numpy array of the maps
          labels: (N, 6) numpy array of the plane parameters
        """
        self.data_files = os.listdir('data_dir') # find all data files, in numpy format
        sort(self.data_files)
        labels = np.load('labels.npy') # N x 6 nd array

        # images = -images # depth range is -1.0 - 0.0, revert it to 0.0 - 1.0
        # total_images = images.transpose(0, 3, 1, 2)
        # self.total_images = torch.from_numpy(total_images).float()
        self.total_labels = torch.from_numpy(labels).float()
        
        self.len = labels.shape[0]
    
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        image = np.load(self.data_files[index]) # depth map
        image.reshape((1, 200, 200))
        image = torch.from_numpy(-image).float()
        label = self.total_labels[index]

        # return image and label
        return image, label

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [17]:
import cv2 as cv

dir = '/content/drive/My Drive/data/Q4A_data/training_set/'

def load_img(path):
  img = cv.imread(path)
  return img[:200, :200, :1]

train_images = []

for i in range(64):
  filename = "{}img_{:03d}_train.png".format(dir, i)
  train_images.append(load_img(filename))

print("All training images loaded!")

train_images = np.array(train_images)
train_labels = np.load(dir + 'Q4A_positions_train.npy')
train_labels = train_labels[:1000]
train_labels = train_labels.reshape(-1, 3)
doubled_train_labels = np.empty((train_labels.shape[0], train_labels.shape[1] * 2))
doubled_train_labels[:, :3] = train_labels
doubled_train_labels[:, 3:6] = train_labels
train_labels = doubled_train_labels

train_data = DepthmapSet(train_images, train_labels)

# dir = '/content/drive/My Drive/data/Q4A_data/testing_set/'

# eval_images = []

# for i in range(10):
#   filename = "{}img_{:03d}_test.png".format(dir, i)
#   eval_images.append(load_img(filename))

# print("All evaluating images loaded!")
  
# eval_images = np.asarray(eval_images)
# eval_labels = np.load(dir + 'Q4A_positions_test.npy')[:100]

# test_data = DepthmapSet(eval_images, eval_labels)

All training images loaded!


In [0]:
import torch
import torch.nn as nn

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# output of the network will be a1, b1, c1, a2, b2, c3. We should try to enforce
# them to satisfy perpendicular condition???! Can we define a custom loss function?
# input will just be a NxN depth map.

class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=6, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False) # 1 is in channel, in this case it's 1 (a depth map)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes) # num_classes: x1, y1, z1, x2, y2, z2 -> 6 values

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)


def resnet34(pretrained=False, progress=True, **kwargs):
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet50(pretrained=False, progress=True, **kwargs):
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet101(pretrained=False, progress=True, **kwargs):
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress,
                   **kwargs)


def resnet152(pretrained=False, progress=True, **kwargs):
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress,
                   **kwargs)


def resnext50_32x4d(pretrained=False, progress=True, **kwargs):
    r"""ResNeXt-50 32x4d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _resnet('resnext50_32x4d', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def resnext101_32x8d(pretrained=False, progress=True, **kwargs):
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)


def wide_resnet50_2(pretrained=False, progress=True, **kwargs):
    r"""Wide ResNet-50-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet50_2', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def wide_resnet101_2(pretrained=False, progress=True, **kwargs):
    r"""Wide ResNet-101-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet101_2', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)

In [0]:
def save_checkpoint(checkpoint_path, model, optimizer):
    # state_dict: a Python dictionary object that:
    # - for a model, maps each layer to its parameter tensor;
    # - for an optimizer, contains info about the optimizer’s states and hyperparameters used.
    state = {
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

In [0]:
def custom_loss(x, y):
    mse_loss = nn.MSELoss()
    loss1 = mse_loss(x, y) * 3
    loss2 = (x[0] * x[3] + x[1] * x[4] + x[2] * x[3]) ** 2 # enforce perpendicular
    totloss = loss1 + loss2
    return totloss

def train_save(epoch, save_interval=1000, log_interval=100):
    model.train()  # set training mode
    iteration = 0
    mse_loss = nn.MSELoss()
    for ep in range(epoch):
        # for batch_idx, (data, target) in enumerate(train_data):
        for data, target in trainloader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = mse_loss(output, target) # x, y, z mse
#             if (loss.item() < 1):
#               print("target:", target)
#               print("predict:", output)
              
            loss.backward()
            optimizer.step()
            if iteration % log_interval == 0:
                print('Train Epoch: {}\tLoss: {:.6f}'.format(
                    ep, loss.item()))
            # different from before: saving model checkpoints
            if iteration % save_interval == 0 and iteration > 0:
                save_checkpoint('drive/My Drive/ckpts/cs336hw2-%i.pth' % iteration, model, optimizer)
            iteration += 1
    
    # save the final model
    save_checkpoint('drive/My Drive/ckpts/drivecs336hw2-%i.pth' % iteration, model, optimizer)

In [24]:
# now train the neural network
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
print(device)

model = wide_resnet50_2().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# create minibatch
indices = list(range(len(train_data)))
np.random.shuffle(indices)
from torch.utils.data.sampler import SubsetRandomSampler
train_sampler = SubsetRandomSampler(indices)
print(train_data[0][0].shape)

trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=64)

train_save(1000, save_interval=1000, log_interval=10)

cuda
torch.Size([1, 200, 200])
Train Epoch: 0	Loss: 16.258787
Train Epoch: 1	Loss: 13.880904
Train Epoch: 2	Loss: 11.784348
Train Epoch: 3	Loss: 10.944756
Train Epoch: 4	Loss: 10.271719
Train Epoch: 5	Loss: 8.624243
Train Epoch: 6	Loss: 6.277220


KeyboardInterrupt: ignored

In [0]:
# run testing!
def test():
    model.eval()  # set evaluation mode
    test_loss = 0
    correct = 0
    # mse_loss = nn.MSELoss()
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = custom_loss(output, target)
            print(loss, data.shape)

# create a new model
model = wide_resnet50_2().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# load from the final checkpoint
load_checkpoint('cs336hw2-10500.pth', model, optimizer)

indices = list(range(len(test_data)))
from torch.utils.data.sampler import SubsetRandomSampler
test_sampler = SubsetRandomSampler(indices)
testloader = torch.utils.data.DataLoader(train_data,
                   sampler=test_sampler, batch_size=100)

# should give you the final model accuracy
test()

In [0]:
# test a single image
import cv2 as cv
from torchvision import models, transforms
from torch.autograd import Variable

def predict(model, filename):
  img = cv.imread(filename)
  print(img.shape)
  img = img / 255.0
  preprocess = transforms.Compose([
   transforms.ToTensor()
  ])
  img_tensor = preprocess(img)
  img_tensor.unsqueeze_(0) # turn image into a batch
  img_tensor = img_tensor.float()
  img_variable = Variable(img_tensor)
  prediction = model(img_variable)
  # print(prediction)
  return prediction

model = resnet18().to('cpu')
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# load from the final checkpoint
load_checkpoint('cs336hw2-500.pth', model, optimizer)
print(predict(model, '/content/drive/My Drive/data/Q4A_data/training_set/img_000_train.png'))

labels = np.load('/content/drive/My Drive/data/Q4A_data/training_set/Q4A_positions_train.npy')
print(labels[0])

model loaded from cs336hw2-500.pth
(480, 640, 3)
tensor([[-0.0303, -0.4224,  7.4414]], grad_fn=<AddmmBackward>)
[-0.30336161 -0.26201127  4.1024477 ]
